In [4]:
import pandas as pd
from itertools import groupby
sortings_sigs = pd.read_pickle("../Benchmark-Models/DeepLoc2/data_files/multisub_ninesignals.pkl")
hou = pd.read_csv("../datasets/final/hou_testset.csv")

compartment_cols = [
    "Membrane", "Cytoplasm", "Nucleus", "Extracellular", "Cell membrane",
    "Mitochondrion", "Plastid", "Endoplasmic reticulum",
    "Lysosome/Vacuole", "Golgi apparatus", "Peroxisome"
]
def combine_compartments(row):
    return ";".join([col for col in compartment_cols if row[col] == 1])
sortings_sigs["Compartments"] = sortings_sigs.apply(combine_compartments, axis=1)
sortings_sigs = sortings_sigs.drop(compartment_cols, axis=1)

sortings_sigs = sortings_sigs.rename({"Sequence": "sequence"}, axis=1)
temp = sortings_sigs.merge(
        hou, how="inner", left_on="ACC", right_on="uniprot_id", 
        suffixes=["_dl2", "_hou"])
print(len(temp))
temp = temp[temp.sequence_hou == temp.sequence_dl2].reset_index(drop=True)
print(len(temp))
temp = temp.drop(["Kingdom", "Partition", "sequence_dl2"], axis=1)
temp.Types.value_counts()

def get_signal_ranges(s):
    ranges = []
    types = []
    start=0
    end=0
    for i, group in groupby(s):
        len_signal = len(list(group))
        end = start+len_signal-1
        if i != 0:
            ranges.append((start, end))
            types.append(s[start])
        start = end+1
    return ranges

def get_signal_type(s):
    types = []
    for i, group in groupby(s):
        if i != 0:
            types.append(i)
    return types

temp["range"] = temp.ANNOT.apply(get_signal_ranges)
temp["signal_type"] = temp.ANNOT.apply(get_signal_type)
temp[["Types", "signal_type"]].set_index("Types").to_dict()
sig_dict = {1.0:"SP", 2.0:"TM", 3.0:"MT", 6.0:"NLS", 7.0:"NES", 8.0:"PTS", 9.0:"GPI"}
temp["signal_type"] = temp.signal_type.apply(lambda l: [sig_dict[i] for i in l])

temp[["uniprot_id", "range", "signal_type"]].to_csv("hou_sorting_signals.csv", index=False)

191
184


In [3]:
sortings_sigs.merge(
        hou, how="inner", left_on="ACC", right_on="uniprot_id", 
        suffixes=["_dl2", "_hou"])

,ACC,ANNOT,Unnamed: 0,Kingdom,Partition,sequence_dl2,Types,Compartments,uniprot_id,ensembl_id,level1,level2,level3,sequence_hou
0,O75489,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...",80,Metazoa,4,MAAAAVARLWWRGILGASALTRGTGRPSVLLLPVRRESAGADTRPT...,MT,Membrane;Mitochondrion,O75489,ENSG00000213619,mitochondria,mitochondria,mitochondria,MAAAAVARLWWRGILGASALTRGTGRPSVLLLPVRRESAGADTRPT...
1,Q96BY9,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",85,Metazoa,4,MAAACGPGAAGYCLLLGLHLFLLTAGPALGWNDPDRMLLRDVKALT...,SP,Membrane;Endoplasmic reticulum,Q96BY9,ENSG00000133872,endoplasmic-reticulum,endoplasmic-reticulum,endomembrane-system,MAAACGPGAAGYCLLLGLHLFLLTAGPALGWNDPDRMLLRDVKALT...
2,P55263,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 6.0, ...",96,Metazoa,1,MAAAEEEPKPKKLKVEAPQALRENILFGMGNPLLDISAVVDKDFLD...,NLS,Nucleus,P55263,ENSG00000156110,cytosol;nucleoplasm,cytosol;nucleus,cytosol;nucleus,MAAAEEEPKPKKLKVEAPQALRENILFGMGNPLLDISAVVDKDFLD...
3,Q9NPA0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",157,Metazoa,4,MAAALWGFFPVLLLLLLSGDVQSSEVPGAAAEGSGGSGVGIGDRFK...,SP_TM,Membrane;Endoplasmic reticulum,Q9NPA0,ENSG00000134153,endoplasmic-reticulum,endoplasmic-reticulum,endomembrane-system,MAAALWGFFPVLLLLLLSGDVQSSEVPGAAAEGSGGSGVGIGDRFK...
4,P49411,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...",207,Metazoa,0,MAAATLLRATPHFSGLAAGRTFLLQGLLRLLKAPALPLLCRGLAVE...,MT,Mitochondrion,P49411,ENSG00000178952,mitochondria,mitochondria,mitochondria,MTTMAAATLLRATPHFSGLAAGRTFLLQGLLRLLKAPALPLLCRGL...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Q9NVH6,"[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, ...",28052,Metazoa,3,MWYHRLSHLHSRLQDLLKGGVIYPALPQPNFKSLLPLAVHWHHTAS...,MT,Mitochondrion,Q9NVH6,ENSG00000185973,mitochondria,mitochondria,mitochondria,MWYHRLSHLHSRLQDLLKGGVIYPALPQPNFKSLLPLAVHWHHTAS...
187,P07602,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",28059,Metazoa,1,MYALFLLASLLGAALAGPVLGLKECTRGSAVWCQNVKTASDCGAVK...,SP,Lysosome/Vacuole,P07602,ENSG00000197746,vesicles;lysosomes,vesicles,endomembrane-system,MYALFLLASLLGAALAGPVLGLKECTRGSAVWCQNVKTASDCGAVK...
188,P02724,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",28095,Metazoa,0,MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTND...,SP_TM,Membrane;Cell membrane,P02724,ENSG00000170180,plasma-membrane,plasma-membrane,plasma-membrane,MYGKIIFVLLLSEIVSISALSTTEVAMHTSTSSSVTKSYISSQTND...
189,Q9Y2I2,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",28155,Metazoa,2,MYLSRFLSIHALWVTVSSVMQPYPLVWGHYDLCKTQIYTEEGKVWD...,SP_GPI,Membrane;Cell membrane,Q9Y2I2,ENSG00000162631,plasma-membrane,plasma-membrane,plasma-membrane,MYLSRFLSIHALWVTVSSVMQPYPLVWGHYDLCKTQIYTEEGKVWD...
